In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
# 코드 가독성을 높여주는 보조 함수

def argmax(vec):
    # argmax를 파이썬 정수현으로 반환
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# 순방향 알고리즘을 위해 수치적으로 안정적인 방법으로 로그 합 지수승을 계산
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1,-1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
# 모델 생성
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)
        
        # LSTM의 출력을 태그 공간으로 대응
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_sizse)
        
        # 전이 매개변수 행렬
        # i,j 성분은 i에서 j로 변할 때의 점수
        self.transitions = nn.Parameter(torch.rand(self.tagset_size, self.tagset_size))
        
        # 이 두 코드는 시작 태그로 전이하지 않고, 정지 태그에서부터 전이하지 않도록 강제한다.
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2)
               torch.randn(2, 1, self.hidden_dim // 2))
    
    def _forward_alg(self, feats):
        # 분배 함수를 계산하기 위해 순방향 알고리즘 수행
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG는 모든 점수를 갖고 있다.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        
        # 자동으로 역전파 되도록 변수로 감싼다.
        forward_var = init_alphas
        
        # 문장의 각 성분을 반복 처리
        for feat in feats:
            alphas_t = [] # 현재 시점의 순방향 텐서
            for next_tag in range(self.tagset_size):
                # 이전의 태그와 상관없이 배출 점수 전파
                emit_score = feat[next_tag].view(1,-1).expand(1, self.tagset_size)
                # trans_score의 i번째 성분은 i로부터 next_tag로 전이할 점수
                trans_score = self.transitions[next_tag].view(1,-1)
                # next_tag_var의 i번째 성분은 로그-합-지수승을 계산하기 전
                # i에서 next_tag로 가는 간선의 값
                next_tag_var = forward_var + trans_score + emit_score
                # 이 태그의 순방향 변수는 모든 점수들의 로그-합-지수승 계산 결과
                alphas_t.append(log_sum_exp(next_tag_var),view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transmitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
    
    def _score_sentence(self, feats, tags):
        # 주어진 태그 순열에 점수를 매긴다.
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tag[i+1], tags[i]] + feat[tags[i+1]]
        score = score + self.transitaions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
      
    def _viterbi_decode(self, feats):
        backpointers = []
        
        # 비터비 변수를 로그 공간 상에 초기화
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        
        # i단계의 forwar_var는 i-1 단계의 비터비 변수를 갖고 있다.
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []
            
            for next_tag in range()